# SHAP VALUES STUDY

In [2]:
!pip install plotly
!pip install boto3==1.19.12
!pip install s3fs
!pip install lightgbm
!pip install shap
!pip install catboost
!pip install plotly s3fs darts shap lightgbm minepy dcor deap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 32.0 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.3.2 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
  Using cached boto3-1.19.12-py3-none-any.whl.metadata (6.4 kB)
  Using cached botocore-1.22.12-py3-none-any.whl.metadata (5.6 kB)
  Using cached jmespath-0.10.0-py2.py3-none-any.whl.metadata (8.0 kB)
  Using cached s3transfer-0.5.2-py3-none-any.whl.metadata (1.7 kB)
Using cached boto3-1.19.12-py3-none-any.whl (131 kB)
Using cached botocore-1.22.12-py3-none-any.whl (8.1 MB)
Using cached jmespath-0.10.0-py2.py3-none-any.whl (24 kB)
Using cached s3transfer-0.5.2-py3-none-any.whl (79 kB)
  Attempting uninstall: jmespath
    Found existing installation: jmespath 1.0.1
    Uninstalling jmespath-1.0.1:
      Successfully uninstalled jmespath-1.0.1
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.19
    Uninstalling botocore-1.34.19:
      Successfully uninstalled

#### Imports

In [4]:
# General
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import os
import numpy as np
# import xlsxwriter
import datetime
import boto3
import s3fs

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV

# Plots
import matplotlib.pyplot as plt

#Warnings
import warnings
warnings.filterwarnings("ignore")


import darts
from darts import TimeSeries
from darts.utils.timeseries_generation import (
    gaussian_timeseries,
    linear_timeseries,
    sine_timeseries,
)

from darts.metrics import mape, smape, mae
from darts.dataprocessing.transformers import Scaler
from darts.utils.timeseries_generation import datetime_attribute_timeseries

from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import RandomForestRegressor

import lightgbm

from darts.models import LightGBMModel

from darts.models import LightGBMModel, RandomForest, LinearRegressionModel
from darts.utils.statistics import check_seasonality, plot_acf, plot_residuals_analysis

from darts.explainability.shap_explainer import ShapExplainer
import pickle
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from darts.models import LinearRegressionModel, LightGBMModel, RandomForest
from calendar import month_name as mn
import os

import shap


#### Paths

#### Read data

In [5]:
cabin = 'Economy'
haul = 'SH'

model_file_path = os.path.join('targets_model', f"best_tuned_mae_model_{cabin}_{haul}_df_LightGBMModel.pkl")
with open(model_file_path, 'rb') as model_file:
    model = pickle.load(model_file)
    
scaler_file_path = os.path.join('targets_model', f"future_scaler_{cabin}_{haul}_df.pkl")
with open(scaler_file_path, 'rb') as scaler_file:
    scaler = pickle.load(scaler_file)
    
# Access the trained LightGBM model
lgb_model = model.model
feature_names = model.lagged_feature_names

daily_data = pd.read_csv('daily_aggregation.csv')
daily_data = daily_data[daily_data['start_date']>='2023-01-01'].drop(columns=['Unnamed: 0'])
variables = [
    'pun_100_punctuality_satisfaction',
    "bkg_200_journey_preparation_satisfaction",
    "pfl_100_checkin_satisfaction",
    "pfl_200_security_satisfaction",
    "pfl_300_lounge_satisfaction",
    "pfl_500_boarding_satisfaction",
    "ifl_300_cabin_satisfaction",
    "ifl_200_flight_crew_annoucements_satisfaction",
    "ifl_600_wifi_satisfaction",
    "ifl_500_ife_satisfaction",
    "ifl_400_food_drink_satisfaction",
    "ifl_100_cabin_crew_satisfaction",
    "arr_100_arrivals_satisfaction",
    "con_100_connections_satisfaction",
    "loy_200_loyalty_programme_satisfaction",
    "img_310_ease_contact_phone_satisfaction",
    "load_factor",
    "otp15_takeoff",
    "NPS_weighted"
]

# daily_data = daily_data[(daily_data['cabin_in_surveyed_flight']==cabin) & (daily_data['haul']==haul)]
daily_data['insert_date_ci'] = '2024-07-15'

In [ ]:
daily_data

In [6]:
def process_dataframe(df):
    df.drop(columns=['pun_100_punctuality_satisfaction', 'inm_400_issues_response_satisfaction'], inplace=True)
    # Agrupar y procesar los datos
    grouped_dfs = {}
    features = {}
    for group_name, group_data in df.groupby(['cabin_in_surveyed_flight', 'haul']):
        cabin_value, haul_value = group_name
        group_df = group_data.copy()
        group_df_name = f'{cabin_value}_{haul_value}_df'
        
        # Identificar las columnas de características
        satisfaction_cols = [col for col in df.columns if col.endswith('_satisfaction')]
        otp_cols = ['otp15_takeoff']
        features_cols = satisfaction_cols + ['load_factor'] + otp_cols
        cols_to_keep = ['insert_date_ci', 'start_date','end_date','cabin_in_surveyed_flight', 'haul'] + features_cols + ['NPS_weighted']

        # Filtrar las columnas en el grupo y actualizar el diccionario de características
        grouped_df = group_df[cols_to_keep]
        features[group_df_name] = features_cols
        grouped_dfs[group_df_name] = grouped_df

    # Reconstruir el DataFrame original
    df = pd.concat(grouped_dfs.values())
    df.reset_index(drop=True, inplace=True)

    return df, grouped_dfs, features

# Aplicar la función a cada DataFrame y almacenar los resultados en las variables correspondientes
day_predict_df, day_predict_df_grouped_dfs, features_cols = process_dataframe(daily_data)

In [7]:
satisfaction_cols = [col for col in day_predict_df.columns if col.endswith('_satisfaction')]
otp_cols = ['otp15_takeoff']
features_cols = satisfaction_cols + ['load_factor'] + otp_cols

In [9]:
import pandas as pd
import numpy as np
import pickle
import os
import pandas as pd
from darts.timeseries import TimeSeries
import os
import pickle
import lightgbm as lgb
import numpy as np
import pandas as pd
import pickle
import os

def compute_shap_and_prediction(row, key, features_cols, K):
    """
    Computes SHAP values and the predicted NPS for a given row.
    
    Parameters:
    - row_df: The DataFrame row for which to compute SHAP values and prediction.
    - key: The key identifying the specific model and scaler to use.
    - features_cols: List of column names representing features used by the model.
    
    Returns:
    - A tuple containing SHAP values as a dictionary and the predicted NPS.
    """
    # Logic to prepare the row for SHAP value computation and prediction
    aux_nps_ts = TimeSeries.from_series(pd.Series([0]))
    aux_row = pd.DataFrame(0, index=[0], columns=row.columns)
    row_df = pd.concat([aux_row, row]).reset_index(drop=True)
    
    # Load the pre-trained model and scaler
    best_tuned_model_dataframe_path = os.path.join('targets_model', f"best_tuned_dataframe_{key}.pkl")
    with open(best_tuned_model_dataframe_path, 'rb') as dataframe_file:
        best_tuned_model = pickle.load(dataframe_file)
    
    future_scaler_path = os.path.join('targets_model', f"future_scaler_{key}.pkl")
    with open(future_scaler_path, 'rb') as scaler_file:
        future_scaler = pickle.load(scaler_file)
    
    future_covariates_ts = TimeSeries.from_dataframe(row_df[features_cols])[-1:]
    future_covariates_ts_scaled = future_scaler.transform(future_covariates_ts)
    
    model_file_path = os.path.join('targets_model', f"best_tuned_mae_model_{key}_{best_tuned_model['model_name']}.pkl")
    with open(model_file_path, 'rb') as model_file:
        model = pickle.load(model_file)
    # Determine the total number of trees and prepare for virtual ensemble
    lgb_model = model.model.booster_
    total_trees = lgb_model.num_trees()
    start_point = total_trees // 2

    predictions = []
    for start in range(start_point, total_trees, K):
        end = min(start + K, total_trees)  # Ensure we do not go beyond total_trees
        pred_value = lgb_model.predict(future_covariates_ts_scaled.pd_dataframe(), start_iteration=start, num_iteration=end - start)
        predictions.append(pred_value)
        
    # Calculate mean and uncertainty of the predictions
    predictions_mean = np.mean(predictions)
    prediction_uncertainty = np.std(predictions)
    
    # Compute SHAP values and prediction
    shap_explain = ShapExplainer(model=model)
    shap_explained = shap_explain.explain(aux_nps_ts, foreground_future_covariates=future_covariates_ts_scaled)
    shap_explanation = shap_explained.get_shap_explanation_object(horizon=1)

    shap_values = shap_explanation[0].values
    base_value = shap_explanation[0].base_values
    pred_value = base_value + shap_values.sum()
    feature_names=[]
    for feat in shap_explanation.feature_names:
        name = [f for f in features_cols if f in feat]
        feature_names.append(name[0])
    
    
    # Convert SHAP values to a dictionary and adjust the logic based on your ShapExplainer
    shap_values_dict = {f"{feature}_nps": value for feature, value in zip(feature_names, shap_values)}
    shap_values_dict["out_prob_base"] = base_value,
    shap_values_dict["out_prob_nps"] = pred_value,
    shap_values_dict["predictions_nps"] = predictions_mean
    shap_values_dict["uncertainty_nps"] = prediction_uncertainty,
    
    # print(row_df.loc[1,features_cols])
    
    shap_explanation = shap.Explanation(values=shap_values, 
                                 base_values=base_value, 
                                 data=np.array(row_df.loc[1,features_cols].values.flatten().tolist()), 
                                 feature_names=shap_explanation.feature_names)
        
    
    return shap_values_dict, shap_explanation, shap_explain, shap_explained
        


# Initialize a dictionary to store the augmented DataFrames
augmented_dfs = {}
explanations = {}

for key in day_predict_df_grouped_dfs.keys():
    # Initialize a list to collect augmented rows
    augmented_rows = []
    explanations[key]={}
    
    n = len(day_predict_df_grouped_dfs[key])
    for index in range(n):
        # Access the row by its index using .iloc
        row_df = day_predict_df_grouped_dfs[key].iloc[[index]]

        # Compute SHAP values and predicted NPS here...
        # Assuming `compute_shap_and_prediction` is a function you'd implement
        # This function should return SHAP values as a dict and the predicted NPS
        shap_values, explanations[key][index], shap_explain, shap_explained = compute_shap_and_prediction(row_df, key, features_cols, K=5)
        
        # For each feature, add its SHAP value to the row
        for feature_name, shap_value in shap_values.items():
            row_df[f'{feature_name}'] = shap_value

        # Append the augmented row to the list
        augmented_rows.append(row_df)
        

    # Concatenate all augmented rows to form the complete augmented DataFrame
    augmented_dfs[key] = pd.concat(augmented_rows).reset_index(drop=True)

# `augmented_dfs` now contains the augmented DataFrames with SHAP values and predictions
augmented_dfs

{'Business_LH_df':     insert_date_ci  start_date    end_date cabin_in_surveyed_flight haul  \
 0       2024-07-15  2023-01-01  2023-01-01                 Business   LH   
 1       2024-07-15  2023-01-02  2023-01-02                 Business   LH   
 2       2024-07-15  2023-01-03  2023-01-03                 Business   LH   
 3       2024-07-15  2023-01-04  2023-01-04                 Business   LH   
 4       2024-07-15  2023-01-05  2023-01-05                 Business   LH   
 ..             ...         ...         ...                      ...  ...   
 532     2024-07-15  2024-06-16  2024-06-16                 Business   LH   
 533     2024-07-15  2024-06-17  2024-06-17                 Business   LH   
 534     2024-07-15  2024-06-18  2024-06-18                 Business   LH   
 535     2024-07-15  2024-06-19  2024-06-19                 Business   LH   
 536     2024-07-15  2024-06-20  2024-06-20                 Business   LH   
 
      bkg_200_journey_preparation_satisfaction  pfl_100_

# Uncertainty propagation

## Systematic error from the models